In [ ]:
debug = True

In [ ]:
import csv
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import json
import numpy as np
import random
import os

In [ ]:
tf.__version__
seed = 18
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(random)

In [ ]:
names = []

with os.scandir('test/') as entries:
    for entry in entries:
        names.append("test/"+entry.name)

In [ ]:
def load(name):
    X = []
    Y = []
    with open(name) as json_file:
        data = json.load(json_file) 
        l = len(data['train'])
        i = 0
        for t in data['train']:
            i += 1
            
            cX = np.zeros((30, 30, channels))
            cY = np.zeros((30, 30, channels))
            
            cin = np.array(t['input'])
            cinr = np.full((30, 30), -1)
            cinr[:cin.shape[0], :cin.shape[1]] = cin
            cX[:, :, 0] = cinr
            #! Clone?
            cX[:, :, 1] = cinr
            
            cout = np.array(t['output'])
            coutr = np.full((30, 30), -1)
            coutr[:cout.shape[0], :cout.shape[1]] = cout
            cY[:, :, 0] = coutr
            #! Clone?
            cY[:, :, 1] = coutr
            X.append(cX)
            Y.append(cY)
            
        for t in data['test']:
            i += 1
            
            cX = np.zeros((30, 30, channels))
            cY = np.zeros((30, 30, channels))
            
            cin = np.array(t['input'])
            cinr = np.full((30, 30), -1)
            cinr[:cin.shape[0], :cin.shape[1]] = cin
            cX[:, :, 0] = cinr
            #! Clone?
            cX[:, :, 1] = cinr
            
            X.append(cX)

    return [np.array(X), np.array(Y)]

In [ ]:
if debug:
    import matplotlib.pyplot as plt

    def plot(inp):
        plt.imshow(inp[:, :, 0].reshape(30, 30))
        plt.colorbar()
        plt.show()

    def plotf(inp):
        plt.imshow(inp)
        plt.colorbar()
        plt.show()

In [ ]:
def loss_f(x, y):
    return tf.reduce_mean(tf.square(x[:, :, :, 0]-y[:, :, :, 0]))

def train(model, ctask, epochs, iterations, channels, verbose=False, validate=False, wholetask=None):
    trainer = tf.keras.optimizers.Adam()

    origx = ctask[0]
    x = origx.copy()
    y = ctask[1]

    mask = np.zeros((x.shape[0], 30, 30, channels))
    mask[:, :, :, 1] = 1
    xl2 = origx*mask

    for k in range(epochs):
        x = origx.copy()
        with tf.GradientTape() as g:
            for i in tf.range(iterations):
                dx = model(x)
                x = x+(dx*1)
                x = x*(1-mask)+xl2
            loss = loss_f(x, y)
            grads = g.gradient(loss, model.weights)
            #grads = [g/(tf.norm(g)+1e-8) for g in grads]
            trainer.apply_gradients(zip(grads, model.weights))

    return model

def generate(model, cin, channels, iterations):
    mask = np.zeros((cin.shape[0], 30, 30, channels))
    mask[:, :, :, 1] = 1
    xl2 = cin*mask
    
    for i in range(iterations):
        dx = model(cin)
        cin = cin+(dx*1)
        cin = cin*(1-mask)+xl2
        if i%10 == 0:
            pass
            #toprint = cin.numpy()[0]
            #plot(toprint)
    
    toprint = cin.numpy()
    return toprint

In [ ]:
def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred

def solve(taskname):
    models = []
    modelcount = 5

    ntasks = 5
    channels = 18
    filters = 3
    perceptionsize = 3
    neurons = 128
    epochs = 200
    iterations = 25

    ctask = load(taskname)
    cin = [ctask[0][0:-1], ctask[1][0:]]
    ctest = [ctask[0][-1]]
    
    if debug:
        plot(ctask[0][0])
        plot(ctask[1][0])
        plot(ctest[0][0])

    outs = []

    for i in range(modelcount):
        model = tf.keras.Sequential([
            tf.keras.layers.DepthwiseConv2D((perceptionsize, perceptionsize), input_shape=(30, 30, channels), strides=[1, 1], padding="same", depth_multiplier=filters, activation="relu"),
            tf.keras.layers.Conv2D(neurons, 1, activation="relu"),
            tf.keras.layers.Conv2D(channels, 1, activation=None, kernel_initializer=tf.zeros_initializer)
        ])
        model = train(model, cin, epochs, iterations, channels, False, False, ctask)
        out = generate(model, np.array([ctest[0]]), channels, iterations)[0]
        outs.append(out)
        plot(out)

    outs = np.array(outs)
    fout = np.clip(np.round(np.mean(outs, axis=0)), -1, 9)
    if debug:
        plot(fout)

    final = fout[:, :, 0]
    x = np.argmin(final[0, :])
    y = np.argmin(final[:, 0])
    cropped = final[:y, :x]
    finalout = flattener(cropped.astype(int).tolist())
    if debug:
        plotf(cropped)
        print(finalout)
    return finalout

In [ ]:
with open('submission.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile, delimiter=',')
    writer.writerow(['output_id','output'])
    for name in names:
        writer.writerow([name, solve(name)])